## Librerias

In [748]:
from IPython.display import display, Markdown
import pandas as pd
import random
from datetime import datetime, timedelta
from typing import List, Tuple

In [749]:
!pip install faker
import faker
fake = faker.Faker()

^C
ERROR: Operation cancelled by user


In [ ]:
!pip install psycopg2-binary
!pip install sqlalchemy
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

## Coneccion a la base de datos

In [ ]:
# Connect to the database and populate with fake data

host = "localhost"
port = 5432
user = "catedra"
password = "S3cret"
database = "postgres"
driver = "postgresql"

pg = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, password, host, port, database))

Session = sessionmaker(bind=pg)

## Mercadopago.SQL

### Correr desde archivo

In [ ]:
# correr mercadopago.sql
with open("mercadopago.sql") as f:
    sql = f.read()
    pg.execute(text(sql))

#### Vaciar la base de datos

In [ ]:
delete_all = """
DROP SCHEMA public CASCADE;
CREATE SCHEMA public;
GRANT ALL ON SCHEMA public TO public;
"""

pg.execute(delete_all)

#### Crear Tablas en la base de datos

In [ ]:
create_tables = """
CREATE TABLE Clave (
    clave_uniforme VARCHAR(50) PRIMARY KEY,
    alias VARCHAR(50) NOT NULL,
    esVirtual BOOLEAN NOT NULL
);

CREATE TABLE Usuarios (
    clave_uniforme VARCHAR(50) PRIMARY KEY,
    CUIT VARCHAR(50),
    email VARCHAR(50),
    nombre VARCHAR(50),
    apellido VARCHAR(50),
    username VARCHAR(50),
    password VARCHAR(50),
    saldo FLOAT,
    fecha_alta DATE,
    FOREIGN KEY (clave_uniforme) REFERENCES Clave(clave_uniforme)
);

CREATE TABLE CuentaBancaria (
    clave_uniforme VARCHAR(50) PRIMARY KEY,
    banco VARCHAR(50),
    FOREIGN KEY (clave_uniforme) REFERENCES Clave(clave_uniforme)
);

CREATE TABLE ProveedorServicio (
    clave_uniforme VARCHAR(50) PRIMARY KEY,
    nombre_empresa VARCHAR(50),
    categoria_servicio VARCHAR(50),
    fecha_alta DATE,
    FOREIGN KEY (clave_uniforme) REFERENCES Clave(clave_uniforme)
);

CREATE TABLE Tarjeta (
    numero VARCHAR(50) PRIMARY KEY,
    vencimiento DATE,
    cvv INTEGER,
    CU VARCHAR(50),
    FOREIGN KEY (CU) REFERENCES Clave(clave_uniforme),
    CHECK (CU IS NOT NULL)
);

-- Add trigger on insert to check if cu is virtual, if not, raise exception

CREATE OR REPLACE FUNCTION check_cu_virtual() 
RETURNS TRIGGER AS $$
BEGIN
    IF (SELECT esVirtual FROM Clave WHERE clave_uniforme = NEW.CU) = FALSE THEN
        RAISE EXCEPTION 'CU is not virtual';
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER check_cu_virtual_trigger
BEFORE INSERT ON Tarjeta
FOR EACH ROW
EXECUTE FUNCTION check_cu_virtual();

CREATE TABLE Transaccion (
    codigo SERIAL PRIMARY KEY,
    CU_Origen VARCHAR(50),
    CU_Destino VARCHAR(50),
    monto FLOAT,
    fecha DATE,
    descripcion VARCHAR(50),
    estado VARCHAR(50),
    es_con_tarjeta BOOLEAN,
    numero VARCHAR(50),
    interes FLOAT,
    FOREIGN KEY (CU_Origen) REFERENCES Clave(clave_uniforme),
    FOREIGN KEY (CU_Destino) REFERENCES Clave(clave_uniforme),
    FOREIGN KEY (numero) REFERENCES Tarjeta(numero)
);

-- Create trigger to check if user has enough balance to make transaction without card
CREATE OR REPLACE FUNCTION check_balance()
RETURNS TRIGGER AS $$
BEGIN
    IF (SELECT saldo FROM Usuarios WHERE clave_uniforme = NEW.CU_Origen) < NEW.monto THEN
        RAISE EXCEPTION 'Not enough balance';
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER check_balance_trigger
BEFORE INSERT ON Transaccion
FOR EACH ROW
WHEN (NEW.es_con_tarjeta = FALSE)
EXECUTE FUNCTION check_balance();


CREATE TABLE Rendimiento (
    id SERIAL PRIMARY KEY,
    fecha_pago DATE,
    comienzo_plazo DATE,
    fin_plazo DATE,
    TNA FLOAT,
    monto FLOAT
);

CREATE TABLE RendimientoUsuario (
    clave_uniforme VARCHAR(50),
    id INTEGER,
    PRIMARY KEY (clave_uniforme, id),
    FOREIGN KEY (clave_uniforme) REFERENCES Clave(clave_uniforme),
    FOREIGN KEY (id) REFERENCES Rendimiento(id)
);

CREATE TABLE TransaccionTarjeta (
    codigo INTEGER,
    numero VARCHAR(50),
    PRIMARY KEY (codigo, numero),
    FOREIGN KEY (numero) REFERENCES Tarjeta(numero),
    FOREIGN KEY (codigo) REFERENCES Transaccion(codigo)
);

"""

pg.execute(create_tables)

## Crear datos de prueba

### Crear identificadores - DNI, CUIT, CVU/CBU, ALIAS

In [1098]:
def random_dni():
    return str(random.randint(10**7, 10**8 - 1))


def random_cuit():
    return (
        str(random.randint(10, 99))
        + "-"
        + random_dni()
        + "-"
        + str(random.randint(0, 9))
    )


def random_cu():
    return (
        "0"
        + str(random.randint(0, 999))
        + "0"
        + str(random.randint(0, 9999))
        + str(random.randint(0, 9))
        + str(random.randint(10**12, 10**13 - 1))
    )


def random_alias():
    return ".".join(fake.words(nb=3))


cu = random_cu()
alias = random_alias()
cuit = random_cuit()

display(cu, alias, cuit)


'04390410921721037937345'

'professor.mind.term'

'17-41113552-3'

### Crear usuarios de prueba

In [1099]:
def insert_clave_uniforme(
        cu: str,
        alias: str,
        esVirtual: bool
    ) -> None:
    """Inserta una clave uniforme en la base de datos

    Args:
        cu (str): Clave Uniforme
        alias (str): Alias
        esVirtual (bool): Es virtual o no
    """

    pg.execute("INSERT INTO Clave VALUES (%s, %s, %s)", (cu, alias, esVirtual))


def create_identifiers() -> Tuple[str, str, str]:
    """Crea identificadores aleatorios

    Returns:
        Tuple[str, str, str]: CU, Alias, CUIT
    """

    cu = random_cu()
    alias = random_alias()
    cuit = random_cuit()

    return cu, alias, cuit


def create_usuario() -> Tuple[str, str, str, str, str, str, str, str]:
    """Crea un usuario aleatorio

    Returns:
        Tuple[str, str, str, str, str, str, str, str]: CVU, Alias, CUIT, Email, Nombre, Apellido, Username, Password
    """

    cvu, alias, cuit = create_identifiers()
    insert_clave_uniforme(cvu, alias, True)
    pg.execute(
        "INSERT INTO Usuarios VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
        (
            cvu,
            cuit,
            (email := fake.email()),
            (first_name := fake.first_name()),
            (last_name := fake.last_name()),
            (user_name := fake.user_name()),
            (password := fake.password()),
            0,
            fake.date_this_century(),
        ),
    )
    return cvu, alias, cuit, email, first_name, last_name, user_name, password


def create_cuenta_bancaria() -> Tuple[str, str]:
    """Crea una cuenta bancaria aleatoria

    Returns:
        Tuple[str, str]: CU, Alias
    """
    cvu, alias, _ = create_identifiers()
    insert_clave_uniforme(cvu, alias, False)
    pg.execute("INSERT INTO CuentaBancaria VALUES (%s, %s)", (cvu, fake.company()))
    return cvu, alias


def create_proveedor_servicio() -> Tuple[str, str]:
    """Crea un proveedor de servicio aleatorio

    Returns:
        Tuple[str, str]: CU, Alias
    """
    cvu, alias, _ = create_identifiers()
    insert_clave_uniforme(cvu, alias, False)
    pg.execute(
        "INSERT INTO ProveedorServicio VALUES (%s, %s, %s, %s)",
        (cvu, fake.company(), fake.word(), fake.date_this_century()),
    )
    return cvu, alias


def add_tarjeta(
        cvu : str,
        number : str,
        cvv : str,
        vencimiento : str
    ):
    """Agrega una tarjeta a la base de datos

    Args:
        cvu (str): CVU del usuario al que pertenece la tarjeta
        number (str): Número de tarjeta
        cvv (str): CVV
        vencimiento (str): Fecha de vencimiento
    """

    pg.execute(
        "INSERT INTO Tarjeta VALUES (%s, %s, %s, %s)", (number, vencimiento, cvv, cvu)
    )


def create_tarjeta(cvu : str) -> Tuple[str, str, str]:
    """Crea una tarjeta aleatoria

    Args:
        cvu (str): CVU del usuario al que pertenece la tarjeta

    Returns:
        Tuple[str, str, str]: Número, CVV, Vencimiento
    """
    
    add_tarjeta(
        cvu,
        (num := fake.credit_card_number()),
        (cvv := fake.credit_card_security_code()),
        (vencimiento := str(fake.date_between(start_date="-1y", end_date="+1y"))),
    )
    return num, cvv, vencimiento


cvu1, alias1, cuit1, email1, first_name1, last_name1, user_name1, password1 = create_usuario()
cvu2, alias2, cuit2, email2, first_name2, last_name2, user_name2, password2 = create_usuario()

cvu_serv1, alias_serv1 = create_proveedor_servicio()
cvu_cuenta1, alias_cuenta1 = create_cuenta_bancaria()

display(
    Markdown(
        f"""
# Usuarios
- {first_name1} {last_name1} - {email1} - {user_name1} - {password1} | {cuit1} - {cvu1} - {alias1}
- {first_name2} {last_name2} - {email2} - {user_name2} - {password2} | {cuit2} - {cvu2} - {alias2}

# Proveedores de Servicio
- {alias_serv1} - {cvu_serv1}

# Cuentas Bancarias
- {alias_cuenta1} - {cvu_cuenta1}
"""
    )

)


# Usuarios
- Barbara Thomas - austinsmith@example.org - stephaniehogan - $dWs0TP12y | 38-17724867-2 - 01800815411454799198944 - strong.rate.take
- Brandon Simmons - andersonholly@example.net - diazmatthew - V)6R#ddE&! | 41-57496073-0 - 04190182422751276284708 - another.quickly.administration

# Proveedores de Servicio
- bed.by.production - 06000927979224297220437

# Cuentas Bancarias
- back.large.politics - 09900382787940710365443


In [1100]:
num_tar1, cvv_tar1, venc_tar1 = create_tarjeta(cvu1)
num_tar1, cvv_tar1, venc_tar1

('3502672493399201', '570', '2025-01-08')

### Probar hacer una transacción entre 2 usuarios uno sin saldo

In [1101]:
# Try to insert a transaction without enough balance

try:
    pg.execute(
        """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (
            cvu1,
            cvu2,
            100,
            fake.date_this_century(),
            "Transferencia",
            "Pendiente",
            False,
            None,
            0,
        ),
    )
    
except Exception as e:
    if "Not enough balance" in str(e):
        display(Markdown("## Not enough balance"))

## Not enough balance

### Función para hacer la transacción debitada de un usuario

In [1102]:
def create_transaccion_debit_between_users(
        amount : float,
        description : str,
        sender_cu : str = "",
        reciever_cu : str = "",
        sender_alias : str = "",
        reciever_alias : str = "",
    ) -> int:
    """Debita una cantidad de dinero de un usuario y la transfiere a otro

    Args:
        sender (str): CVU del usuario que envía el dinero
        reciever (str): CVU del usuario que recibe el dinero
        amount (float): Cantidad de dinero
        description (str): Descripción de la transacción

    Returns:
        int: Código de la transacción
    """

    # Check for balance first
    if pg.execute("SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (sender_cu,)).fetchone()[0] < amount:
        raise Exception("Not enough balance")

    if not (sender_cu or sender_alias):
        raise Exception("Invalid Sender - No CU or Alias")
    
    if not (reciever_cu or reciever_alias):
        raise Exception("Invalid Reciever - No CU or Alias")

    if (sender_cu != ""):
        # Check if sender exists
        if not pg.execute("SELECT clave_uniforme FROM Usuarios WHERE clave_uniforme = %s", (sender_cu,)).fetchone():
            raise Exception("Invalid Sender - Not Found")
        # Check if the sender is virtual
        if not pg.execute("SELECT esVirtual FROM Clave WHERE clave_uniforme = %s", (sender_cu,)).fetchone()[0]:
            raise Exception("Invalid Sender - Not Virtual")

    if (sender_alias != "" and sender_cu is ""):
        sender_cu = pg.execute("SELECT clave_uniforme FROM Clave WHERE alias = %s", (sender_alias,)).fetchone()[0]
        if not sender_cu:
            raise Exception("Invalid Sender - Alias not found")
    
    if (reciever_alias != "" and reciever_cu is ""):
        reciever_cu = pg.execute("SELECT clave_uniforme FROM Clave WHERE alias = %s", (reciever_alias,)).fetchone()[0]
        if not reciever_cu:
            raise Exception("Invalid Reciever - Alias not found")

    pg.execute(
        """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (
            sender_cu,
            reciever_cu,
            amount,
            datetime.now(),
            description,
            fake.word(),
            False,
            None,
            None,
        ),
    )

    # El siguiente código se ejecuta directamente como un trigger en la base de datos

    # pg.execute(
    #     "UPDATE Usuarios SET saldo = saldo - %s WHERE clave_uniforme = %s",
    #     (amount, sender),
    # )

    # pg.execute(
    #     "UPDATE Usuarios SET saldo = saldo + %s WHERE clave_uniforme = %s",
    #     (amount, reciever),
    # )

    return pg.execute("SELECT codigo FROM Transaccion ORDER BY codigo DESC LIMIT 1").fetchone()[0]

# Try again to insert a transaction without enough balance

try:
    create_transaccion_debit_between_users(100, "Transferencia", sender_cu=cvu1, reciever_cu=cvu2)
except Exception as e:
    if "Not enough balance" in str(e):
        display(Markdown("## Not enough balance"))

<>:39: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:44: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:39: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:44: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/5j/7b3w3dw95qdcll87vxc3yn300000gn/T/ipykernel_911/2075508739.py:39: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (sender_alias != "" and sender_cu is ""):
/var/folders/5j/7b3w3dw95qdcll87vxc3yn300000gn/T/ipykernel_911/2075508739.py:44: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (reciever_alias != "" and reciever_cu is ""):


## Not enough balance

### Depositar dinero en una cuenta desde una cuenta de bancaria

In [1103]:
def create_transaccion_deposit(
    user: str, cbu_cuenta_bancaria: str, amount: float, description: str
) -> Tuple[str, float]:
    """Deposita dinero en la cuenta de un usuario

    Args:
        user (str): CVU del usuario
        amount (float): Cantidad de dinero
        description (str): Descripción de la transacción

    Returns:
        Tuple[str, float]: Código de la transacción, Saldo final
    """

    pg.execute(
        """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (
            cbu_cuenta_bancaria,
            user,
            amount,
            fake.date_this_century(),
            description,
            fake.word(),
            False,
            None,
            None,
        ),
    )

    # El siguiente código se ejecuta directamente como un trigger en la base de datos
    
    # pg.execute(
    #     "UPDATE Usuarios SET saldo = saldo + %s WHERE clave_uniforme = %s",
    #     (amount, user),
    # )

    return (
        str(
            pg.execute(
                "SELECT codigo FROM Transaccion ORDER BY codigo DESC LIMIT 1"
            ).fetchone()[0]
        ),
        float(
            pg.execute(
                "SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (user,)
            ).fetchone()[0]
        ),
    )


In [1104]:
codigo_transaccion, saldo_final = create_transaccion_deposit(
    cvu1, cvu_cuenta1, 1000, "Depósito"
)

display(
    Markdown(
        f"## Transacción de depósito\n- Código: {codigo_transaccion}\n- Saldo final: {saldo_final}"
    )
)

## Transacción de depósito
- Código: 2
- Saldo final: 1000.0

### Transferir dinero entre dos cuentas ahora si con saldo

In [1105]:
# Transfer money from user 1 to user 2 now that user 1 has enough balance

codigo_transaccion = create_transaccion_debit_between_users(
    100, "Transferencia", sender_cu=cvu1, reciever_cu=cvu2
)

display(Markdown(f"## Transacción de transferencia\n- Código: {codigo_transaccion}"))

# Buscar transacciones y mostrar

pg.execute("SELECT * FROM Transaccion WHERE codigo = %s", (codigo_transaccion,)).fetchone()

## Transacción de transferencia
- Código: 3

(3, '01800815411454799198944', '04190182422751276284708', 100.0, datetime.date(2024, 5, 10), 'Transferencia', 'everyone', False, None, None)

In [1106]:
# Listar nombres de Servicios

nombres = pg.execute("SELECT nombre_empresa FROM ProveedorServicio").fetchall()

display(Markdown(f"## Nombres de servicios\n- {', '.join([n[0] for n in nombres])}"))

sample_nombre = random.choice(nombres)[0]

display(Markdown(f"## Nombre de servicio aleatorio\n- {sample_nombre}"))

## Nombres de servicios
- Taylor LLC

## Nombre de servicio aleatorio
- Taylor LLC

In [1107]:
# Pagar un servicio

def create_transaccion_pay_service(
    user: str, service_name: str, amount: float, description: str
) -> Tuple[str, float]:
    """Paga un servicio

    Args:
        user (str): CVU del usuario
        service_name (str): Nombre del servicio
        amount (float): Cantidad de dinero
        description (str): Descripción de la transacción
    
    Returns:
        Tuple[str, float]: Código de la transacción, Saldo final
    """

    # Check for balance first
    if pg.execute("SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (user,)).fetchone()[0] < amount:
        raise Exception("Not enough balance")

    # Hallar el cbu del servicio por su nombre
    cbu_servicio = pg.execute(
        "SELECT clave_uniforme FROM ProveedorServicio WHERE nombre_empresa = %s",
        (service_name,),
    ).fetchone()[0]

    pg.execute(
        """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
        (
            user,
            cbu_servicio,
            amount,
            fake.date_this_century(),
            description,
            fake.word(),
            False,
            None,
            None,
        ),
    )

    pg.execute(
        "UPDATE Usuarios SET saldo = saldo - %s WHERE clave_uniforme = %s",
        (amount, user),
    )

    return (
        str(
            pg.execute(
                "SELECT codigo FROM Transaccion ORDER BY codigo DESC LIMIT 1"
            ).fetchone()[0]
        ),
        float(
            pg.execute(
                "SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (user,)
            ).fetchone()[0]
        ),
    )

create_transaccion_pay_service(cvu1, sample_nombre, 100, "Pago de servicio")

('4', 700.0)

In [1108]:
# Invertir plata en la fecha y generar rendimientos a futuro


def comenzar_inversion(
    cvu : str,
):  
    """
    CREATE TABLE Rendimiento (
        id SERIAL PRIMARY KEY,
        fecha_pago DATE,
        comienzo_plazo DATE NOT NULL,
        fin_plazo DATE,
        TNA FLOAT,
        monto FLOAT
    );
    """
    # Se pone a invertir el monto total del usuario
    # La fecha del comienzo del plazo es ahora,
    # El fin del plazo es en 1 dia
    # La fecha de pago es el próximo dia hábil
    # La tna es entre 60 y 80
    # El monto es el saldo del usuario

    # Solo se puede tener un rendimiento en un comienzo de plazo dado
    # Si ya hay un rendimiento activo y se va a generar otro, se debe finalizar el anterior

    # Hallar el saldo del usuario
    saldo = pg.execute(
        "SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu,)
    ).fetchone()[0]

    # Hallar si hay un rendimiento activo
    rendimiento_activo = pg.execute(
        "SELECT * FROM Rendimiento WHERE comienzo_plazo = CURRENT_DATE"
    ).fetchone()

    if rendimiento_activo:
        # Finalizar el rendimiento activo
        pg.execute(
            "UPDATE Rendimiento SET fin_plazo = CURRENT_DATE WHERE id = %s",
            (rendimiento_activo[0],),
        )

    # Generar un nuevo rendimiento
    tna = random.uniform(60, 80)
    monto = saldo
    fin_plazo = datetime.now() + timedelta(days=1)
    
    pg.execute(
        """INSERT INTO Rendimiento (comienzo_plazo, fin_plazo, TNA, monto)
        VALUES (%s, %s, %s, %s)""",
        (
            datetime.now(),
            fin_plazo,
            tna,
            monto,
        ),
    )

    # Asociar el rendimiento al usuario
    rendimiento_id = pg.execute(
        "SELECT id FROM Rendimiento ORDER BY id DESC LIMIT 1"
    ).fetchone()[0]

    pg.execute(
        "INSERT INTO RendimientoUsuario VALUES (%s, %s)",
        (cvu, rendimiento_id),
    )

    return rendimiento_id

def pagar_rendimientos_activos_usuario(
    cvu : str
) -> List:
    """
    Paga los rendimientos de un usuario

    Args:
        cvu (str): Clave uniforme del usuario

    Returns:
        List: Lista de los montos pagados
    """

    # Hallar los rendimientos no pagados del usuario, es decir sin fecha de pago
    # JOIN con Rendimiento para hallar los datos del rendimiento

    rendimientos = pg.execute(
        """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
        FROM RendimientoUsuario
        JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
        WHERE clave_uniforme = %s AND pago IS FALSE""",
        (cvu,),
    ).fetchall()
    print(len(rendimientos))

    for rendimiento in rendimientos:
        # Pagar el rendimiento
        updated = pg.execute(
            """
            UPDATE Rendimiento 
            SET fecha_pago = CURRENT_DATE WHERE id = %s
            RETURNING id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
            """,
            (rendimiento[0],),
        ).fetchone()
        print(updated)

        # Añadir el rendimiento generado al saldo del usuario
        # El rendimiento es el monto * (1 + TNA / 100) * (fin_plazo - comienzo_plazo) / 365

        # rendimiento_generado = rendimiento[5] * (1 + rendimiento[4] / 100) * (rendimiento[3] - rendimiento[2]).days / 365
        rendimiento_generado = rendimiento[5]
        rendimiento_generado *= 1 + rendimiento[4] / 100
        rendimiento_generado *= (rendimiento[3] - rendimiento[2]).days / 365

        pg.execute(
            "UPDATE Usuarios SET saldo = saldo + %s WHERE clave_uniforme = %s",
            (rendimiento_generado, cvu),  
        )

    if not rendimientos:
        return []

    # Refetch los rendimientos para mostrar by r[0] for r in rendimientos
    updated_rendimientos = pg.execute(
        """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
        FROM RendimientoUsuario
        JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
        WHERE clave_uniforme = %s AND Rendimiento.id IN %s""",
        (cvu, tuple([r[0] for r in rendimientos])),
    ).fetchall()

    return updated_rendimientos

In [1109]:
rendimiento_id = comenzar_inversion(cvu1)

display(Markdown(f"## Rendimiento comenzado\n- ID: {rendimiento_id}"))

rendimientos = pagar_rendimientos_activos_usuario(cvu1)

display(
    Markdown("## Rendimientos pagados" + "\n".join([f"""
- ID: {r[0]}
- Fecha de pago: {r[1]}
- Comienzo del plazo: {r[2]}
- Fin del plazo: {r[3]}
- TNA: {r[4]}
- Monto Invertido: {r[5]}
- Monto Pagado: {r[5] * (1 + r[4] / 100) * (r[3] - r[2]).days / 365}
""" for r in rendimientos])
    )
)

## Rendimiento comenzado
- ID: 1

1
(1, datetime.date(2024, 5, 10), datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 79.77730186659493, 700.0)


## Rendimientos pagados
- ID: 1
- Fecha de pago: 2024-05-10
- Comienzo del plazo: 2024-05-10
- Fin del plazo: 2024-05-11
- TNA: 79.77730186659493
- Monto Invertido: 700.0
- Monto Pagado: 3.447783871414149


In [1110]:
# Crear un nuevo usuario 3
# depositar 1000,
# comenzar un rendimiento
# interrumpirlo transfiriendole 500 al usuario 1 (Agregar un trigger para que cuando se cree una transacción, si el usuario tiene un rendimiento activo, se finalice y se cree uno nuevo con el monto restante pero que no se pague)
# continuar el rendimiento y pagar los rendimientos

# TLDR; Hacer una transferencia con un rendimiento activo y ver que pasa


cvu3, alias3, cuit3, email3, first_name3, last_name3, user_name3, password3 = create_usuario()

# Mostrar saldo cvu3
display(pg.execute("SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu3,)).fetchone()[0])

create_transaccion_deposit(cvu3, cvu_cuenta1, 1000, "Depósito")

# Mostrar saldo cvu3 después del depósito
display(pg.execute("SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu3,)).fetchone()[0])

rendimiento_id = comenzar_inversion(cvu3)

display(
    pg.execute(
        """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
    FROM RendimientoUsuario
    JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
    WHERE clave_uniforme = %s""",
        (cvu3,),
    ).fetchall()
)


0.0

1000.0

[(2, None, datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 78.05212666518727, 1000.0)]

In [1111]:
# Checkear si el usuario tiene un rendimiento activo, es decir fecha de fin_plazo es posterior a la fecha de la transaccion
# Joinear RendimientoUsuario con Rendimiento para obtener el id del rendimiento activo
display(
    pg.execute(
        """
        SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
        FROM RendimientoUsuario 
        JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id 
        WHERE clave_uniforme = %s -- AND fin_plazo > (SELECT fecha FROM Transaccion WHERE CU_Origen = %s ORDER BY fecha DESC LIMIT 1)
        """,
        (cvu3, cvu3),
    ).fetchall()
)

[(2, None, datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 78.05212666518727, 1000.0)]

In [1112]:
create_transaccion_debit_between_users(500, "Transferencia", sender_cu=cvu3, reciever_cu=cvu1)
# Mostrar saldo cvu3 después de la transferencia
display(
    pg.execute(
        "SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu3,)
    ).fetchone()[0]
)

500.0

In [1113]:
display(
    pg.execute(
        """
        SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
        FROM RendimientoUsuario 
        JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id 
        WHERE clave_uniforme = %s
        """,
        (cvu3,),
    ).fetchall()
)

[(2, None, datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 78.05212666518727, 1000.0),
 (3, None, datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 78.05212666518727, 500.0)]

In [1114]:
rendimientos = pagar_rendimientos_activos_usuario(cvu3)
# Mostrar saldo cvu3 después de pagar los rendimientos
display(
    pg.execute(
        "SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu3,)
    ).fetchone()[0]
)

display(
    Markdown(
        "## Rendimientos pagados"
        + "\n".join(
            [
                f"""
- ID: {r[0]}
- Fecha de pago: {r[1]}
- Comienzo del plazo: {r[2]}
- Fin del plazo: {r[3]}
- TNA: {r[4]}
- Monto Invertido: {r[5]}
- Monto Pagado: {r[5] * (1 + r[4] / 100) * (r[3] - r[2]).days / 365}
"""
                for r in rendimientos
            ]
        )
    )
)

2
(2, datetime.date(2024, 5, 10), datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 78.05212666518727, 1000.0)
(3, datetime.date(2024, 5, 10), datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 78.05212666518727, 500.0)


507.3172106848707

## Rendimientos pagados
- ID: 2
- Fecha de pago: 2024-05-10
- Comienzo del plazo: 2024-05-10
- Fin del plazo: 2024-05-11
- TNA: 78.05212666518727
- Monto Invertido: 1000.0
- Monto Pagado: 4.878140456580473


- ID: 3
- Fecha de pago: 2024-05-10
- Comienzo del plazo: 2024-05-10
- Fin del plazo: 2024-05-11
- TNA: 78.05212666518727
- Monto Invertido: 500.0
- Monto Pagado: 2.4390702282902366


In [1115]:
pg.execute(
    """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
    FROM RendimientoUsuario
    JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
    WHERE clave_uniforme = %s""",
    (cvu3,),
).fetchall()

[(2, datetime.date(2024, 5, 10), datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 78.05212666518727, 1000.0),
 (3, datetime.date(2024, 5, 10), datetime.date(2024, 5, 10), datetime.date(2024, 5, 11), 78.05212666518727, 500.0)]

In [1116]:
# Obtener todos los registros de rendimientos de cvu3

rendimientos_pendientes = pg.execute(
    """SELECT Rendimiento.id, fecha_pago, comienzo_plazo, fin_plazo, TNA, monto
    FROM RendimientoUsuario
    JOIN Rendimiento ON RendimientoUsuario.id = Rendimiento.id
    WHERE clave_uniforme = %s AND fecha_pago IS NULL""",
    (cvu3,),
).fetchall()

display(
    Markdown(
        f"## Rendimientos Pendientes de {alias3}"
        + "\n".join(
            [
                f"""
- ID: {r[0]}
- Fecha de pago: {r[1]}
- Comienzo del plazo: {r[2]}
- Fin del plazo: {r[3]}
- TNA: {r[4]}
- Monto Invertido: {r[5]}
"""
                for r in rendimientos_pendientes
            ]
        )
    )
)

## Rendimientos Pendientes de so.assume.single

In [1117]:
# Mostrar usuario cvu2

pg.execute("SELECT * FROM Usuarios WHERE clave_uniforme = %s", (cvu3,)).fetchone()

('06290601458791732651415', '31-99782146-7', 'zoecooper@example.net', 'Lawrence', 'Ray', 'tracy43', 'I#CNwTq0_0', 507.3172106848707, datetime.date(2006, 6, 6))

In [1118]:
# Crear un nuevo usuario, y pagarle a cvu1 con una tarjeta

cvu4, alias4, cuit4, email4, first_name4, last_name4, user_name4, password4 = (
    create_usuario()
)

create_tarjeta(cvu4)

valor = 100

In [1119]:
cvu4, alias4, cuit4, email4, first_name4, last_name4, user_name4, password4

('01430216785517305071878',
 'interview.relationship.some',
 '13-68957297-6',
 'feliciatate@example.org',
 'Stephen',
 'Diaz',
 'emmasimmons',
 'v0uGcFS)#w')

In [1120]:
pg.execute("SELECT * FROM Tarjeta WHERE CU = %s", (cvu4,)).fetchone()

('30341176197244', datetime.date(2023, 9, 23), 858, '01430216785517305071878')

In [1121]:
pg.execute("SELECT * FROM Tarjeta").fetchall()

[('3502672493399201', datetime.date(2025, 1, 8), 570, '01800815411454799198944'),
 ('30341176197244', datetime.date(2023, 9, 23), 858, '01430216785517305071878')]

In [1122]:
pg.execute(
    """INSERT INTO Transaccion (CU_Origen, CU_Destino, monto, fecha, descripcion, estado, es_con_tarjeta, numero, interes)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
    (
        cvu4,
        cvu1,
        valor,
        fake.date_this_century(),
        "Transferencia",
        "Pendiente",
        True,
        num_tar1,
        8,
    ),
)

codigo_transaccion = pg.execute("SELECT codigo FROM Transaccion ORDER BY codigo DESC LIMIT 1").fetchone()[0]

pg.execute(
    """
        INSERT INTO TransaccionTarjeta VALUES (%s, %s)
    """,
    (codigo_transaccion, num_tar1),
)

display(Markdown(f"## Transacción de transferencia con tarjeta\n- Código: {codigo_transaccion}"))

## Transacción de transferencia con tarjeta
- Código: 7

### Pagar a un usuario de MercadoPago por su alias

In [1123]:
# Pagar a un usuario de MercadoPago por su alias

# Saldo de cvu1 y alias4 antes de la transacción

display(
    f"Saldo de cvu1: {cvu1} antes de la transacción",
    pg.execute(
        "SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu1,)
    ).fetchone()[0]
)

display(
    f"Saldo de alias4: {alias4} antes de la transacción",
    pg.execute(
        "SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu4,)
    ).fetchone()[0]
)

create_transaccion_debit_between_users(100, "Pago de servicio", sender_cu=cvu1, reciever_alias=alias4)

# Saldo de cvu1 y alias4 despues de la transacción

display(
    f"Saldo de cvu1: {cvu1} antes de la transacción",
    pg.execute(
        "SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu1,)
    ).fetchone()[0],
)

display(
    f"Saldo de alias4: {alias4} antes de la transacción",
    pg.execute(
        "SELECT saldo FROM Usuarios WHERE clave_uniforme = %s", (cvu4,)
    ).fetchone()[0],
)

'Saldo de cvu1: 01800815411454799198944 antes de la transacción'

1303.4477838714142

'Saldo de alias4: interview.relationship.some antes de la transacción'

0.0

'Saldo de cvu1: 01800815411454799198944 antes de la transacción'

1203.4477838714142

'Saldo de alias4: interview.relationship.some antes de la transacción'

100.0